In [10]:
from datetime import datetime, timedelta

import pytz
from IPython.display import Image, display
from langchain_groq import ChatGroq

In [11]:
from langchain_core.tools import tool
from langchain.tools import BaseTool, StructuredTool, tool
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
# from langchain.pydantic_v1 import BaseModel, Field
from pydantic import BaseModel, Field
from langchain_core.tools import ToolException

@tool
def SetReminderTool(activity_name: str, start_time_hour:int, start_time_minutes: int, duration:int, run_manager: Optional[CallbackManagerForToolRun] = None, 
    ) -> str:
        """useful for when you need to set a reminder. Do not set a reminder until user agrees on an activity , time and duration """
        return f"Reminder set for {activity_name} for {start_time_hour}hrs {start_time_minutes}mins for a duration of {duration} minutes"

In [12]:
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda

from langgraph.prebuilt import ToolNode


def handle_tool_error(state) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
            ToolMessage(
                content=f"Error: {repr(error)}\n please fix your mistakes.",
                tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }


def create_tool_node_with_fallback(tools: list) -> dict:
    return ToolNode(tools).with_fallbacks(
        [RunnableLambda(handle_tool_error)], exception_key="error"
    )


def _print_event(event: dict, _printed: set, max_length=1500):
    current_state = event.get("dialog_state")
    if current_state:
        print("Currently in: ", current_state[-1])
    message = event.get("messages")
    if message:
        if isinstance(message, list):
            message = message[-1]
        if message.id not in _printed:
            msg_repr = message.pretty_repr(html=True)
            if len(msg_repr) > max_length:
                msg_repr = msg_repr[:max_length] + " ... (truncated)"
            print(msg_repr)
            # _printed.add(message.id)


In [38]:
from typing import TypedDict, Literal
from langchain_core.messages import AnyMessage
from response_templates.supervisor_response import SupervisorResponse

# Conversation State to hold all user interaction details
class ConversationState(TypedDict):
    exchange: int
    conversation_history: list[AnyMessage]
    preferred_activities: list[str]
    user_input: str
    supervisor_response: SupervisorResponse
    flow: str
    messages: list[AnyMessage]

In [39]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig


# class Frienn:
#     def __init__(self, runnable: Runnable):
#         self.runnable = runnable

#     def __call__(self, state: ConversationState, config: RunnableConfig):
#         while True:
#             configuration = config.get("configurable", {})
#             passenger_id = configuration.get("passenger_id", None)
#             state = {**state, "user_info": passenger_id}
#             result = self.runnable.invoke(state)
            
#             if not result.tool_calls and (
#                 not result.content
#                 or isinstance(result.content, list)
#                 and not result.content[0].get("text")
#             ):
#                 messages = state["messages"] + [("user", "Respond with a real output.")]
#                 state = {**state, "messages": messages}
#             else:
#                 break
#         return {"messages": result}

In [40]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import tools_condition

builder = StateGraph(State)


# # Define nodes: these do the work
# builder.add_node("assistant", Assistant(part_1_assistant_runnable))
# builder.add_node("tools", create_tool_node_with_fallback(part_1_tools))
# # Define edges: these determine how the control flow moves
# builder.add_edge(START, "assistant")
# builder.add_conditional_edges(
#     "assistant",
#     tools_condition,
# )
# builder.add_edge("tools", "assistant")

# # The checkpointer lets the graph persist its state
# # this is a complete memory for the entire graph.
# memory = MemorySaver()
# part_1_graph = builder.compile(checkpointer=memory)


# from IPython.display import Image, display

# try:
#     display(Image(part_1_graph.get_graph(xray=True).draw_mermaid_png()))
# except Exception:
#     # This requires some extra dependencies and is optional
#     pass

NameError: name 'State' is not defined

In [45]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage
from response_templates.conversation_state import ConversationState
from utils import exchanges_pretty, get_current_time_ist, fetch_user_preferences, get_current_time_ist_30min_lag

class ChatEngine:
    def __init__(self, llm:Runnable):
        self.llm = llm
        self.bot_char_prompt = self.get_frienn_char_prompt()
        self.conversation_history = []
        self.exchange = 0
        self.flow = "activity_suggestion"

    def get_frienn_char_prompt(self):
        
        base_char_prompt = f'''You are Frienn, a kind and empathetic virtual companion designed by Friendly, designed to suggest activities to improve users mood and follow up on the activities.

        Behavior Guidelines:

        Be empathetic, respectful, and friendly.
        Respond with brief, short and clear sentences.
        If the user is felling low offering thoughtful suggestions or encouragement if not just chat like a friend
        Never provide medical, legal, or financial advice.

        '''

        return base_char_prompt

    def get_activity_suggestion_guidelines(self):

        return '''Activity Suggestion Guidelines:

                1. Prioritize the user's preferred activities; otherwise, suggest a suitable one.
                2. Avoid digital engagement activities or games.
                3. Consider the user's time and location when suggesting activities, including appropriate duration.
                4. Ask if they want to do it now or later, rounding the suggested time.
                5. Keep choices and questions minimal to avoid overwhelming the user.
                6. Confirm the activity and time before finalizing.
                7. If not immediate, set a reminder using below command in the response:
                    <set_reminder> {chosen_activity} at {start_time} until {end_time}</set_reminder>
                    if multiple repeat the above line for each activity and time combination
                8. After setting the reminder, try to end the conversation.

                '''
                
    def get_reminder_details(self):
        return f'''Walking at {get_current_time_ist()}'''



    def generate_response(self, user_input, preferred_activities):

        conversation_history_pretty = exchanges_pretty(self.conversation_history)
        print("You:", user_input)
        
        if self.flow == "activity_suggestion":
            if self.exchange == 0:
                chat_prompt_msgs = [
                    SystemMessage(self.get_frienn_char_prompt()),
                    SystemMessage(self.get_activity_suggestion_guidelines()),
                    SystemMessage("Introduce yourself briefly and naturally before suggesting an activity"),
                    HumanMessage(user_input)
                     ]
            else: 
                chat_prompt_msgs = [
                SystemMessage(self.get_frienn_char_prompt()),
                SystemMessage(self.get_activity_suggestion_guidelines()),
                SystemMessage(f"Activities preferred by user: {preferred_activities}"),
                SystemMessage(f"Current time: {get_current_time_ist()}"),
                SystemMessage(f"Conversation History:<conversation_history>{conversation_history_pretty}</conversation_history>"),
                HumanMessage(user_input)
                            ]

        elif self.flow == "reminder":
            if self.exchange == 0:
                chat_prompt_msgs = [
                    SystemMessage(self.bot_char_prompt),
                    SystemMessage(f"Current time: {get_current_time_ist()}"),
                    SystemMessage(f"Conversation History:<conversation_history>{conversation_history_pretty}</conversation_history>"),
                    SystemMessage(f"Previously you set a reminder for {self.get_reminder_details()}. It’s time! Encourage the user to start their activity with enthusiasm and motivation."),
                ]
            else:
                chat_prompt_msgs = [
                    SystemMessage(self.bot_char_prompt),
                    SystemMessage(f"Current time: {get_current_time_ist()}"),
                    SystemMessage(f"Conversation History:<conversation_history>{conversation_history_pretty}</conversation_history>"),
                    SystemMessage(f"Reminder details: {self.get_reminder_details()}. Motivate the user to complete the activity. If they seem reluctant, suggest small fun modifications to make it more enjoyable else end the conversation."),
                    HumanMessage(user_input)
                ]

        elif self.flow == "follow-up":
            if self.exchange == 0:
                chat_prompt_msgs = [
                    SystemMessage(self.bot_char_prompt),
                    SystemMessage(f"Activities preferred by user: {preferred_activities}"),
                    SystemMessage(f"Current time: {get_current_time_ist()}"),
                    SystemMessage(f"Conversation History:<conversation_history>{conversation_history_pretty}</conversation_history>"),
                    SystemMessage(f"You set a reminder for walking at {get_current_time_ist_30min_lag()}. Check in on the user’s experience—ask if they completed it and how it made them feel."),
                ]
            else:
                chat_prompt_msgs = [
                    SystemMessage(self.bot_char_prompt),
                    SystemMessage(f"Activities preferred by user: {preferred_activities}"),
                    SystemMessage(f"Current time: {get_current_time_ist()}"),
                    SystemMessage(f"Previously you have set a reminder for walking at {get_current_time_ist_30min_lag()}. Now you are checking in on the user. Continue the conversation as friend and end the conversation. Do not suggest more activities if user is feeling better."),
                    SystemMessage(f"Conversation History:<conversation_history>{conversation_history_pretty}</conversation_history>"),
                    HumanMessage(user_input)
                ]


        else:
            
            chat_prompt_msgs = [
                SystemMessage(self.bot_char_prompt),
                SystemMessage(f"Activities preferred by user: {preferred_activities}"),
                SystemMessage(f"Current time: {get_current_time_ist()}"),
                SystemMessage(f"Conversation History:<conversation_history>{conversation_history_pretty}</conversation_history>"),
                HumanMessage(user_input)
            ]

        
        model_response = self.llm.invoke(chat_prompt_msgs)
        print(f"Frienn :", model_response.content)
        self.messages.append(model_response)
        
        self.update_conversation_history(user_input, model_response.content)
        self.exchange += 1
        
    def update_conversation_history(self, user_input, response):
        self.conversation_history.append(HumanMessage(content=user_input))
        self.conversation_history.append(AIMessage(content=response))

    def __call__(self,  conversation_state:ConversationState):
        user_input = conversation_state["user_input"]
        preferred_activities = conversation_state.get("preferred_activities", ["no preferences provided"])
        self.conversation_history = conversation_state.get("conversation_history", [])
        self.messages = conversation_state.get("messages", [])
        self.exchange = conversation_state.get("exchange", 0)
        self.flow =  conversation_state.get("flow", "activity_suggestion")
        
        self.generate_response(user_input, preferred_activities)
        
        return {
            "conversation_history": self.conversation_history,
            "user_input": user_input,
            "exchange": self.exchange,
            "messages": self.messages
        }

    # def __call__(self, state: ConversationState, config: RunnableConfig):
    #     while True:
    #         configuration = config.get("configurable", {})
    #         passenger_id = configuration.get("passenger_id", None)
    #         state = {**state, "user_info": passenger_id}
    #         result = self.runnable.invoke(state)
            
    #         if not result.tool_calls and (
    #             not result.content
    #             or isinstance(result.content, list)
    #             and not result.content[0].get("text")
    #         ):
    #             messages = state["messages"] + [("user", "Respond with a real output.")]
    #             state = {**state, "messages": messages}
    #         else:
    #             break
    #     return {"messages": result}

In [69]:
from typing import TypedDict, Literal
from langchain_core.messages import AnyMessage
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

# Assuming these are defined elsewhere in your project
from supervisor import Supervisor
# from chat_engine import ChatEngine
from reminder_manager import set_activity_reminder
from crisis_handler import crisis_handler
from response_templates.supervisor_response import SupervisorResponse
from response_templates.conversation_state import ConversationState

# Graph Builder Class to manage the state graph and routing
class ConversationGraph:
    def __init__(self,llm_sv, llm_f, frienn_tools):
        self.llm_sv = llm_sv
        self.llm_f = llm_f
        self.frienn_tools = frienn_tools
        self.builder = StateGraph(ConversationState)
        self._add_nodes()
        self._add_edges()
        


    def _add_nodes(self):
        """Add all nodes to the graph"""
        self.builder.add_node("Supervisor", Supervisor(llm=self.llm_sv).get_supervisor_decision)
        self.builder.add_node("Frienn", ChatEngine(llm=self.llm_f))
        self.builder.add_node("crisisHandler", crisis_handler)
        self.builder.add_node("frienn_tools", create_tool_node_with_fallback(self.frienn_tools))

    def _add_edges(self):
        """Define and add edges to the graph"""
        self.builder.add_edge(START, "Supervisor")
        self.builder.add_conditional_edges("Supervisor", self._determine_route)
        self.builder.add_conditional_edges("Frienn",tools_condition,{
        # If it returns 'action', route to the 'tools' node
        "action": "frienn_tools",
        # If it returns '__end__', route to the end
        "__end__": "__end__",
    },)
        self.builder.add_edge("Frienn", END)
        self.builder.add_edge("crisisHandler", END)
        

    def _determine_route(self, conversation_state: ConversationState) -> Literal["Frienn", "crisisHandler"]:
        """Determine the next route based on the supervisor response"""
        supervisor_response = conversation_state.get("supervisor_response")
        picked_route = supervisor_response.pickedRoute

        if picked_route == 'continue_chat':
            return "Frienn"
        elif picked_route == 'crisis_helpline':
            return "crisisHandler"
        # elif picked_route == 'set_reminder':
        #     return "setReminder"
        else:
            return "Frienn"

    def compile(self):
        """Compile the final state graph with memory saver"""
        memory = MemorySaver()
        return self.builder.compile(checkpointer=memory)

In [70]:
from utils import get_llm
llm_sv = get_llm()
llm_f = get_llm().bind_tools([SetReminderTool])


In [77]:
conversation_graph = ConversationGraph(llm_sv, llm_f, frienn_tools=[SetReminderTool]).compile()
# processor = ConversationProcessor(conversation_graph)
cs2 = conversation_graph.update_state(config={'configurable':{'thread_id':"1", 'user_id':"dev-user"}}, values = {'preferred_activities': [ 'walking'], 'flow':'activity_suggestion'})

In [78]:
tutorial_questions = ["hi"]
config = {"configurable": {"thread_id": 12, "user_id": 'user_id'}}

_printed = set()
for question in tutorial_questions:
    events = conversation_graph.stream(
        {"user_input": question}, config, stream_mode="values"
    )
    for event in events:
        _print_event(event, _printed)

# user_input = "hi"
# config = {"configurable": {"thread_id": 121, "user_id": 'user_id'}}
# conversation_graph.invoke({"user_input": user_input}, config)

=====> Supervisor Decision: pickedRoute='continue_chat' reason='initial greeting, no indication of crisis' <=====
You: hi
Frienn : I'm Frienn, nice to meet you. How are you feeling today? Would you like to do something to brighten up your day? Maybe go for a walk or do some stretching?


In [60]:
for e in events:
    print(e)

In [76]:
tutorial_questions = ["good"]
config = {"configurable": {"thread_id": 0, "user_id": 'user_id'}}

_printed = set()
for question in tutorial_questions:
    events = conversation_graph.stream(
        {"user_input": question}, config, stream_mode="debug"
    )
    # for event in events:
    #     _print_event(event, _printed)

In [62]:
tutorial_questions = ["sounds interesting, 8 works"]
config = {"configurable": {"thread_id": 0, "user_id": 'user_id'}}

_printed = set()
for question in tutorial_questions:
    events = conversation_graph.stream(
        {"user_input": question}, config, stream_mode="values"
    )
    for event in events:
        _print_event(event, _printed)

=====> Supervisor Decision: pickedRoute='continue_chat' reason='user is interested in going for a walk and has agreed on a time, no indication of crisis or harmful intentions' <=====
You: sounds interesting, 8 works
Frienn : 


KeyError: 'tools'

In [73]:
user_input = "hi"
config = {"configurable": {"thread_id": 121, "user_id": 'user_id'}}
conversation_graph.invoke({"user_input": user_input}, config)

=====> Supervisor Decision: pickedRoute='continue_chat' reason='user just started the conversation with a greeting and no indication of crisis or harmful intentions' <=====
You: hi
Frienn : I'm Frienn, nice to meet you. How's your day going so far? Want to do something fun?


{'exchange': 2,
 'conversation_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm Frienn, nice to meet you. How's your day going so far? Want to do something fun?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm Frienn, nice to meet you. How's your day going so far? Want to do something fun?", additional_kwargs={}, response_metadata={})],
 'user_input': 'hi',
 'supervisor_response': SupervisorResponse(pickedRoute='continue_chat', reason='user just started the conversation with a greeting and no indication of crisis or harmful intentions')}

In [74]:
user_input = "good"
config = {"configurable": {"thread_id": 121, "user_id": 'user_id'}}
conversation_graph.invoke({"user_input": user_input}, config)

=====> Supervisor Decision: pickedRoute='continue_chat' reason='user responded with a positive sentiment' <=====
You: good
Frienn : That's great to hear. Would you like to go for a walk or do some stretching exercises? We could do it now or later, say around 8:00 AM?


{'exchange': 3,
 'conversation_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm Frienn, nice to meet you. How's your day going so far? Want to do something fun?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm Frienn, nice to meet you. How's your day going so far? Want to do something fun?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='good', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's great to hear. Would you like to go for a walk or do some stretching exercises? We could do it now or later, say around 8:00 AM?", additional_kwargs={}, response_metadata={})],
 'user_input': 'good',
 'supervisor_response': SupervisorResponse(pickedRoute='continue_chat', reason='user responded with a positive sentiment')}

In [75]:
user_input = "ok"
config = {"configurable": {"thread_id": 121, "user_id": 'user_id'}}
conversation_graph.invoke({"user_input": user_input}, config)

=====> Supervisor Decision: pickedRoute='continue_chat' reason="User's input 'ok' indicates a neutral or positive response, and there's no indication of harmful intentions or suicidal tendencies." <=====
You: ok
Frienn : 


KeyError: 'tools'